In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002845287322998047
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 13.52179322171197
200000 25.519562880043402
300000 31.255623528851327
400000 34.5263544988548
500000 32.75608012589111
600000 33.262535751968116
700000 34.6170519423123
800000 33.24569784050386
900000 32.400654765812114
1000000 32.76187821518198
1100000 32.46002043033531
1200000 32.24622768938476
1300000 15.673538531843127
1400000 24.055565368446562
1500000 31.90496958298937
1600000 32.314070297131885
1700000 31.772344327183166
1800000 32.12341485769903
1900000 32.48711016516139
2000000 34.25070206968557
2100000 32.738154649287296
2200000 32.41479231592931
2300000 33.15054684948687
2400000 32.81699964850915
2500000 32.935980937450346
2600000 12.984768032268034
2700000 28.719424324324752
2800000 41.14029872545978
2900000 42.0632396719703
3000000 41.55283270

29700000 28.867622975999314
29800000 30.852337750655817
29900000 31.68154298420541
30000000 31.89161619771066
30100000 31.937291807206886
30200000 31.40674879352571
30300000 32.4470457752859
30400000 33.45805802065739
30500000 32.30999297445564
30600000 31.42220461421913
30700000 12.589096945852207
30800000 33.90015801873189
30900000 33.64132260410405
31000000 32.472115248044155
31100000 20.004274624520846
31200000 23.817975728568634
31300000 32.52724096584644
31400000 32.222384417161436
31500000 32.213745009701576
31600000 31.464018367848535
31700000 32.965530359035625
31800000 31.888581850437717
31900000 30.729679996553855
32000000 30.767947227915936
32100000 29.86268605979133
32200000 7.361751736963028
32300000 30.93333460429876
32400000 30.71431817320932
32500000 30.627443707541673
32600000 31.664615336798658
32700000 32.03182422706599
32800000 30.806648259559108
32900000 31.962746675084965
33000000 32.16955039942973
33100000 32.40367294520407
33200000 19.874386449262275
33300000 2

59500000 30.267839488923823
59600000 16.214917043754184
59700000 23.857687994822456
59800000 29.791490130532406
59900000 28.217032427459696
60000000 29.143867173879322
60100000 27.14041635786129
60200000 27.17315312736797
60300000 26.730003214923997
60400000 23.383176603403584
60500000 11.87178071761232
60600000 26.800365720503688
60700000 27.30808999800768
60800000 27.55121796756497
60900000 27.94500548512989
61000000 28.8095004600737
61100000 28.216726087462646
61200000 29.156984599048037
61300000 13.701015229316672
61400000 27.996619354033346
61500000 30.28492056660468
61600000 29.385880456803864
61700000 30.943561324310078
61800000 29.504067964166698
61900000 30.00947046919067
62000000 30.049720014642286
62100000 29.894135987364304
62200000 13.860473733500433
62300000 25.581162665752792
62400000 28.796470135727418
62500000 28.353021809610375
62600000 29.05063959669495
62700000 27.285484880163317
62800000 27.2550715290784
62900000 26.704118761260453
63000000 16.31024211435995
631000

89200000 23.169042308302917
89300000 26.068480817262984
89400000 8.518058904808553
89500000 27.849684367433643
89600000 25.453397434775184
89700000 27.446462865524875
89800000 27.2702538374004
89900000 27.065503399298528
90000000 28.609772505700832
90100000 25.701951013165992
90200000 28.21103389751243
90300000 28.130710737099427
90400000 25.85753452937663
90500000 27.061193364785964
90600000 25.848573357609116
90700000 11.126316306393749
90800000 22.491753495854084
90900000 25.83986181652709
91000000 22.94697271534845
91100000 25.402990857160525
91200000 23.972655659254656
91300000 12.632652446601433
91400000 18.693467064582986
91500000 25.791905851776995
91600000 23.417652965465155
91700000 25.182421989732315
91800000 23.724575251836022
91900000 18.299919757995973
92000000 18.231015651824908
92100000 26.650872253136683
92200000 25.00321001193883
92300000 28.061060062807535
92400000 27.185580291973142
92500000 26.503512420097497
92600000 27.152819760455284
92700000 26.675583636414007


118200000 22.64316217801242
118300000 22.428588117574215
118400000 23.94462086553025
118500000 22.82810011427679
118600000 23.366849294513905
118700000 23.3219070003386
118800000 21.713473489929648
118900000 21.487594533522298
119000000 20.945932906959982
119100000 21.17637631379372
119200000 6.552758728621054
119300000 21.48072625051976
119400000 21.155855582461445
119500000 20.490327372077072
119600000 7.214194408896333
119700000 19.262115331938126
119800000 20.241823456745824
119900000 20.955364122875807
120000000 11.39942346759736
120100000 17.706542939110815
120200000 20.940084213165065
120300000 20.478518698615883
120400000 21.181165128541654
120500000 21.88569035200815
120600000 22.093122063085858
120700000 22.95706786348382
120800000 22.397205103348057
120900000 22.78129310616409
121000000 23.464992878597016
121100000 22.967143512758277
121200000 22.702943133710804
121300000 22.619841610337648
121400000 21.856217802893838
121500000 21.138167031104345
121600000 20.59125808816814

...

232800000 6.583741843904934
232900000 10.666849158555593
233000000 11.34100420277907
233100000 8.970394491914748
233200000 5.646316749390816
233300000 10.89246547222544
233400000 10.84102407120565
233500000 11.452528986226415
233600000 11.352678262050482
233700000 11.618551593140038
233800000 12.003472474561448
233900000 12.054205811272524
234000000 12.0627053892177
234100000 11.916991461337942
234200000 12.028965269353385
234300000 12.13126650121874
234400000 11.80765880746851
234500000 10.943172175199738
234600000 10.967419710429338
234700000 10.587282729583274
234800000 10.989725863335062
234900000 3.9068753775837006
235000000 10.975986935386995
235100000 10.916087836523916
235200000 10.927522867926566
235300000 4.567361142575842
235400000 9.044046385722583
235500000 10.774232303890495
235600000 10.417453145384863
235700000 8.090889114018692
235800000 6.631928452172787
235900000 10.435243478093085
236000000 10.783604133870837
236100000 11.266731026583605
236200000 11.22106261317696


261800000 9.276873296083329
261900000 9.334195166232453
262000000 9.40104488980819
262100000 9.64459566908296
262200000 10.138499490588478
262300000 8.867724859913702
262400000 9.537628881679918
262500000 9.273165924976565
262600000 9.233043946358258
262700000 9.036190992098918
262800000 8.811419625031256
262900000 7.623733606199408
263000000 4.505975671876282
263100000 8.227069713556823
263200000 8.422548837100285
263300000 8.21644009519418
263400000 8.122561947890333
263500000 8.194243075109094
263600000 2.3796526274768266
263700000 8.029384964826445
263800000 8.283494193141845
263900000 8.351268523241563
264000000 8.233256364006355
264100000 8.261737307124049
264200000 5.666826783232884
264300000 5.80352517901529
264400000 8.727532133841526
264500000 8.731262817954432
264600000 9.016208286589972
264700000 8.920406500994341
264800000 9.402691182102723
264900000 9.501678122874024
265000000 9.321880527827465
265100000 8.916518540782365
265200000 9.249960793815212
265300000 8.7867401149

291100000 4.332389124162769
291200000 6.123585379187144
291300000 5.974668994179756
291400000 6.046606134277892
291500000 5.77489774755508
291600000 5.834158627594032
291700000 5.668507185952097
291800000 5.897001204318978
291900000 1.1042643611661016
292000000 5.943204560210777
292100000 5.734317563295678
292200000 5.777193739411293
292300000 5.709766545847708
292400000 6.037437697334307
292500000 5.952089141403223
292600000 6.053907607031564
292700000 3.8208785879477616
292800000 4.634082735472212
292900000 6.246940851843059
293000000 6.146154097512887
293100000 6.306869466942148
293200000 6.385977604860048
293300000 6.367722085409087
293400000 6.269525383350736
293500000 6.193562469967168
293600000 2.096715635724153
293700000 6.5747997336025445
293800000 6.053400472640669
293900000 5.830033565591966
294000000 5.882435149598756
294100000 5.6015257610849725
294200000 5.53498871493831
294300000 5.57835636420113
294400000 4.321174774652347
294500000 2.75630347270314
294600000 5.43688795

320200000 1.1645666408500455
320300000 3.2418948517151636
320400000 3.296077398153294
320500000 3.360607483321154
320600000 3.3988440741575703
320700000 3.5688068743527075
320800000 3.335906553374314
320900000 3.4239419390705828
321000000 3.4610612231777633
321100000 3.409809693411382
321200000 1.8712371100417506
321300000 2.724872639306833
321400000 3.577163552535447
321500000 3.3919616025984345
321600000 3.1670121517163525
321700000 1.4964107947719336
321800000 3.387998268820356
321900000 3.2827562687730034
322000000 3.41353449444924
322100000 3.2388541390617256
322200000 3.140250081676592
322300000 3.2972815102272928
322400000 3.194162381212576
322500000 3.1550512054644333
322600000 3.0570560358568493
322700000 2.0151762533004263
322800000 1.917622554814439
322900000 3.0463729093272467
323000000 3.2050016851517507
323100000 3.133253505891237
323200000 3.192423586572202
323300000 3.073887548874157
323400000 3.156974464307033
323500000 3.2414192409983573
323600000 3.165970753185786
32

348800000 0.6677484574028062
348900000 0.6511430385199679
349000000 0.655767110893017
349100000 0.625014816792243
349200000 0.6343145623882016
349300000 0.6096641853287658
349400000 0.6025551862981107
349500000 0.5683920899573306
349600000 0.576425371555531
349700000 0.2613678650543365
349800000 0.2720424538188848
349900000 0.4338869947445094
350000000 0.4290567994156738
350100000 0.4053724951896793
350200000 0.40714377618402037
350300000 0.40076801665422385
350400000 0.4114123928037352
350500000 0.38276550926918984
350600000 0.36407986101573636
350700000 0.3414952781577971
350800000 0.34196565488902525
350900000 0.3087854004556848
351000000 0.18131004461916292
351100000 0.22643508079228403
351200000 0.27991764870497915
351300000 0.29520397043162117
351400000 0.29311434792487623
351500000 0.28505923953612844
351600000 0.2773478832066198
351700000 0.28172640445750746
351800000 0.2605778430660241
351900000 0.25318584998937116
352000000 0.25527149599585136
352100000 0.2409081887825562
352